# 📤 Cómo Entregar tu Trabajo

Este notebook te permite entregar tu actividad del **Mapa Logístico Extendido** con un solo clic.

---

## 📋 Antes de entregar

**Asegúrate de:**
- ✅ Haber completado todos los ejercicios en `mapa_logistico_ext.ipynb`
- ✅ Haber ejecutado todas las celdas (para que se vean los resultados)
- ✅ Haber revisado tus respuestas

---

## 🚀 Proceso de Entrega

El proceso es muy simple y **NO requiere crear ninguna cuenta**:

1. **Ingresa tu nombre** en la celda de abajo
2. **Haz clic en "Entregar trabajo"**
3. **¡Listo!** Tu notebook se envía automáticamente

**No necesitas descargar nada ni abrir correo. Todo se hace aquí.**

---

In [ ]:
import json
from js import fetch, console, document, Object, JSON, FormData
from pyodide.ffi import to_js
import asyncio

# IMPORTANTE: Después de desplegar en Vercel, reemplaza esta URL con la tuya
    "SERVER_URL = \"https://chaos-sub.vercel.app/submit\"\n",

async def submit_notebook():
    """Envía el notebook al servidor usando la API de JavaScript"""
    try:
        # Obtener el nombre del estudiante del input HTML
        student_name = document.getElementById("studentName").value.strip()
        
        if not student_name:
            document.getElementById("result").innerHTML = '<div style="color: #d9534f; padding: 10px; background: #f2dede; border-radius: 4px;">❌ Por favor ingresa tu nombre</div>'
            return
        
        # Mostrar mensaje de carga
        document.getElementById("result").innerHTML = '<div style="color: #5bc0de; padding: 10px; background: #d9edf7; border-radius: 4px;">⏳ Enviando tu trabajo...</div>'
        
        # Leer el notebook desde el kernel de Jupyter
        notebook_path = "mapa_logistico_ext.ipynb"
        
        try:
            # Intentar leer el notebook
            with open(notebook_path, 'r') as f:
                notebook_content = json.load(f)
        except FileNotFoundError:
            document.getElementById("result").innerHTML = '<div style="color: #d9534f; padding: 10px; background: #f2dede; border-radius: 4px;">❌ No se encontró el archivo mapa_logistico_ext.ipynb. Asegúrate de haber ejecutado ese notebook primero.</div>'
            return
        
        # Preparar los datos para enviar
        payload = {
            "student_name": student_name,
            "notebook": notebook_content
        }
        
        # Convertir a formato JavaScript
        js_payload = to_js(payload, dict_converter=Object.fromEntries)
        
        # Configurar la petición fetch
        fetch_options = {
            "method": "POST",
            "headers": {
                "Content-Type": "application/json"
            },
            "body": JSON.stringify(js_payload)
        }
        
        # Enviar la petición
        response = await fetch(SERVER_URL, to_js(fetch_options, dict_converter=Object.fromEntries))
        
        # Obtener la respuesta como texto
        response_text = await response.text()
        
        # Verificar el status
        if response.ok:
            # Éxito
            result_data = json.loads(response_text)
            document.getElementById("result").innerHTML = f'''
                <div style="color: #5cb85c; padding: 15px; background: #dff0d8; border-radius: 4px; border-left: 4px solid #5cb85c;">
                    <h3 style="margin-top: 0;">✅ ¡Trabajo entregado exitosamente!</h3>
                    <p><strong>Estudiante:</strong> {student_name}</p>
                    <p><strong>Fecha:</strong> {result_data.get("timestamp", "N/A")}</p>
                    <p style="margin-bottom: 0;">Tu profesor ha recibido tu trabajo. ¡Buen trabajo! 🎉</p>
                </div>
            '''
        else:
            # Error del servidor
            try:
                error_data = json.loads(response_text)
                error_msg = error_data.get("error", "Error desconocido")
            except:
                error_msg = response_text
            
            document.getElementById("result").innerHTML = f'''
                <div style="color: #d9534f; padding: 15px; background: #f2dede; border-radius: 4px; border-left: 4px solid #d9534f;">
                    <h3 style="margin-top: 0;">❌ Error al enviar</h3>
                    <p>{error_msg}</p>
                    <p style="margin-bottom: 0;">Por favor, intenta de nuevo o contacta al profesor.</p>
                </div>
            '''
            
    except Exception as e:
        console.error(f"Error en submit_notebook: {str(e)}")
        document.getElementById("result").innerHTML = f'''
            <div style="color: #d9534f; padding: 15px; background: #f2dede; border-radius: 4px; border-left: 4px solid #d9534f;">
                <h3 style="margin-top: 0;">❌ Error inesperado</h3>
                <p>{str(e)}</p>
                <p style="margin-bottom: 0;">Por favor, intenta de nuevo o contacta al profesor.</p>
            </div>
        '''

print("✅ Celda de configuración cargada correctamente")
print("👇 Ahora ejecuta la celda de abajo para mostrar el formulario de entrega")

In [ ]:
from IPython.display import display, HTML

# Crear el formulario HTML
form_html = """
<div style="max-width: 600px; margin: 20px auto; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Arial, sans-serif;">
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <h2 style="color: white; margin-top: 0; text-align: center;">📝 Entregar Trabajo</h2>
        
        <div style="background: white; padding: 20px; border-radius: 6px; margin-top: 20px;">
            <label for="studentName" style="display: block; margin-bottom: 8px; font-weight: 600; color: #333;">
                Tu nombre completo:
            </label>
            <input 
                type="text" 
                id="studentName" 
                placeholder="Ej: María González" 
                style="width: 100%; padding: 12px; border: 2px solid #e0e0e0; border-radius: 4px; font-size: 16px; box-sizing: border-box;"
            />
            
            <button 
                onclick="submitWork()" 
                style="width: 100%; margin-top: 20px; padding: 14px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border: none; border-radius: 4px; font-size: 16px; font-weight: 600; cursor: pointer; transition: transform 0.2s;"
                onmouseover="this.style.transform='scale(1.02)'" 
                onmouseout="this.style.transform='scale(1)'"
            >
                🚀 Entregar trabajo
            </button>
        </div>
        
        <div id="result" style="margin-top: 20px;"></div>
    </div>
    
    <div style="text-align: center; margin-top: 20px; color: #666; font-size: 14px;">
        <p>💡 <strong>Tip:</strong> Asegúrate de haber ejecutado todas las celdas de tu notebook antes de entregar.</p>
    </div>
</div>

<script>
    async function submitWork() {
        // Llamar a la función Python
        try {
            await pyodide.runPythonAsync(`
                import asyncio
                asyncio.ensure_future(submit_notebook())
            `);
        } catch (error) {
            document.getElementById("result").innerHTML = '<div style="color: #d9534f; padding: 10px; background: #f2dede; border-radius: 4px;">❌ Error: ' + error + '</div>';
        }
    }
</script>
"""

display(HTML(form_html))
print("📋 Formulario listo. Ingresa tu nombre y haz clic en 'Entregar trabajo'")

In [ ]:
# ============================================
# SISTEMA DE ENTREGA DE TRABAJOS
# ============================================

from IPython.display import display, HTML, Javascript
import ipywidgets as widgets
from datetime import datetime
import urllib.parse

def create_download_and_email_system():
    """
    Sistema de entrega sin necesidad de cuentas.
    Los estudiantes descargan su notebook y pueden enviarlo por email.
    """
    
    # =============================================
    # CONFIGURACIÓN
    # =============================================
    
    # TODO: Cambia este email por el tuyo
    PROFESOR_EMAIL = "profesor@uc.cl"
    
    NOTEBOOK_NAME = "mapa_logistico_ext"  # Nombre base del archivo
    SUBJECT_NAME = "Mapa Logístico Extendido"  # Nombre de la actividad
    
    # =============================================
    # INTERFAZ
    # =============================================
    
    # Input nombre
    name_input = widgets.Text(
        placeholder='Ejemplo: Juan Pérez',
        description='Tu nombre:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='450px')
    )
    
    # Botón de descarga
    download_btn = widgets.Button(
        description='📥 Descargar mi trabajo',
        button_style='success',
        icon='download',
        layout=widgets.Layout(width='300px', height='45px'),
        disabled=True
    )
    
    # Output areas
    status_output = widgets.Output()
    email_section = widgets.HTML()
    
    # =============================================
    # LÓGICA
    # =============================================
    
    def on_name_change(change):
        """Habilita botón cuando hay nombre"""
        if change['new'].strip():
            download_btn.disabled = False
            download_btn.button_style = 'success'
        else:
            download_btn.disabled = True
            download_btn.button_style = 'info'
    
    def on_download_click(b):
        """Descarga el notebook y muestra instrucciones de envío"""
        with status_output:
            status_output.clear_output()
            
            student_name = name_input.value.strip()
            if not student_name:
                display(HTML("""
                    <div style='color: #d32f2f; padding: 15px; background-color: #ffebee; 
                                border-radius: 8px; margin-top: 10px; border-left: 5px solid #d32f2f;'>
                        <b>⚠️ Error:</b> Por favor ingresa tu nombre completo.
                    </div>
                """))
                return
            
            # Generar nombre de archivo
            safe_name = student_name.replace(' ', '_').replace('/', '_').replace('\\', '_')
            safe_name = ''.join(c for c in safe_name if c.isalnum() or c in ('_', '-'))
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"{safe_name}_{NOTEBOOK_NAME}_{timestamp}.ipynb"
            
            # JavaScript para descargar
            display(Javascript(f"""
                (function() {{
                    try {{
                        var notebook = Jupyter.notebook;
                        var notebook_json = notebook.toJSON();
                        var blob = new Blob(
                            [JSON.stringify(notebook_json, null, 2)], 
                            {{type: 'application/json'}}
                        );
                        var url = URL.createObjectURL(blob);
                        var a = document.createElement('a');
                        a.href = url;
                        a.download = '{filename}';
                        document.body.appendChild(a);
                        a.click();
                        document.body.removeChild(a);
                        setTimeout(() => URL.revokeObjectURL(url), 100);
                    }} catch (error) {{
                        console.error('Error:', error);
                        alert('Error al descargar. Intenta de nuevo.');
                    }}
                }})();
            """))
            
            # Confirmación
            display(HTML(f"""
                <div style='background: linear-gradient(135deg, #e8f5e9 0%, #c8e6c9 100%); 
                            padding: 20px; border-radius: 10px; margin-top: 15px; 
                            border-left: 5px solid #4caf50; box-shadow: 0 2px 8px rgba(0,0,0,0.1);'>
                    <h3 style='margin: 0 0 15px 0; color: #2e7d32;'>
                        ✅ ¡Archivo descargado!
                    </h3>
                    <div style='background: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>
                        <p style='margin: 5px 0;'><b>📄 Archivo:</b> <code>{filename}</code></p>
                        <p style='margin: 5px 0;'><b>👤 Estudiante:</b> {student_name}</p>
                        <p style='margin: 5px 0;'><b>📁 Ubicación:</b> Carpeta de Descargas</p>
                    </div>
                </div>
            """))
            
            # Crear enlace mailto
            subject = urllib.parse.quote(f"Entrega: {SUBJECT_NAME} - {student_name}")
            body = urllib.parse.quote(f"""Hola Profesor,

Adjunto mi trabajo del {SUBJECT_NAME}.

Nombre: {student_name}
Fecha: {datetime.now().strftime('%d/%m/%Y %H:%M')}

Saludos,
{student_name}
""")
            mailto_link = f"mailto:{PROFESOR_EMAIL}?subject={subject}&body={body}"
            
            # Mostrar sección de email
            email_section.value = f"""
                <div style='background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%); 
                            padding: 25px; border-radius: 12px; margin-top: 20px; 
                            border: 3px solid #2196f3; box-shadow: 0 4px 12px rgba(0,0,0,0.15);'>
                    <h3 style='color: #1565c0; margin-top: 0;'>
                        📧 Paso 2: Enviar por correo
                    </h3>
                    
                    <div style='background: #fff3cd; padding: 15px; border-radius: 8px; 
                                margin: 15px 0; border-left: 4px solid #ffc107;'>
                        <p style='margin: 0; color: #856404; font-size: 15px;'>
                            <b>📂 Busca este archivo en tu carpeta de Descargas:</b><br>
                            <code style='background: white; padding: 4px 10px; border-radius: 4px; 
                                         margin-top: 8px; display: inline-block; color: #d32f2f;'>
                                {filename}
                            </code>
                        </p>
                    </div>
                    
                    <div style='text-align: center; margin: 25px 0;'>
                        <a href="{mailto_link}" target="_blank"
                           style='display: inline-block; background: #2196f3; color: white; 
                                  padding: 15px 40px; border-radius: 8px; text-decoration: none; 
                                  font-weight: bold; font-size: 18px; box-shadow: 0 4px 8px rgba(0,0,0,0.2);
                                  transition: transform 0.2s;'
                           onmouseover="this.style.transform='scale(1.05)'" 
                           onmouseout="this.style.transform='scale(1)'">
                            📧 Abrir correo para enviar
                        </a>
                    </div>
                    
                    <div style='background: white; padding: 15px; border-radius: 8px;'>
                        <p style='margin: 5px 0; font-size: 14px; color: #666;'>
                            <b>Qué pasará al hacer clic:</b>
                        </p>
                        <ol style='margin: 10px 0; color: #666; font-size: 14px;'>
                            <li>Se abrirá tu programa de correo (Gmail, Outlook, etc.)</li>
                            <li>El correo estará pre-llenado con el asunto y mensaje</li>
                            <li>Adjunta el archivo <code>{filename}</code></li>
                            <li>Presiona <b>Enviar</b></li>
                        </ol>
                        <p style='margin: 10px 0 5px 0; font-size: 14px; color: #1976d2;'>
                            <b>💡 También puedes:</b> Enviar manualmente a <code>{PROFESOR_EMAIL}</code>
                        </p>
                    </div>
                    
                    <div style='background: #e8f5e9; padding: 15px; border-radius: 8px; 
                                margin-top: 15px; border-left: 4px solid #4caf50;'>
                        <p style='margin: 0; color: #2e7d32; font-size: 14px;'>
                            ✅ <b>Recuerda:</b> No olvides adjuntar el archivo antes de enviar el correo.
                        </p>
                    </div>
                </div>
            """
    
    # Conectar eventos
    name_input.observe(on_name_change, names='value')
    download_btn.on_click(on_download_click)
    
    # =============================================
    # INSTRUCCIONES
    # =============================================
    
    instructions = widgets.HTML(value=f"""
        <div style='background: linear-gradient(135deg, #fff3cd 0%, #ffe082 100%); 
                    padding: 20px; border-radius: 12px; 
                    border-left: 5px solid #ff9800; margin-bottom: 20px;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);'>
            <h3 style='margin-top: 0; color: #e65100;'>
                📋 Instrucciones de Entrega
            </h3>
            <ol style='margin: 15px 0; color: #663c00; line-height: 1.8; font-size: 15px;'>
                <li>Ingresa tu <b>nombre completo</b> en el campo de abajo</li>
                <li>Haz clic en <b>"Descargar mi trabajo"</b></li>
                <li>El archivo se guardará en tu carpeta de <b>Descargas</b></li>
                <li>Haz clic en el botón de correo que aparecerá</li>
                <li>Adjunta el archivo descargado y presiona <b>Enviar</b></li>
            </ol>
            <div style='background: #e8f5e9; padding: 12px; border-radius: 6px; margin-top: 15px;'>
                <p style='margin: 0; font-size: 14px; color: #2e7d32;'>
                    <b>✅ Sin complicaciones:</b> No necesitas crear ninguna cuenta. 
                    Solo descarga y envía por correo.
                </p>
            </div>
        </div>
    """)
    
    # =============================================
    # LAYOUT
    # =============================================
    
    main_box = widgets.VBox([
        widgets.HTML("""
            <h1 style='color: #1976d2; text-align: center; 
                       margin: 20px 0; font-size: 32px;'>
                📤 Sistema de Entrega de Trabajos
            </h1>
            <p style='text-align: center; color: #666; font-size: 16px; margin-bottom: 30px;'>
                Actividad: <b>""" + SUBJECT_NAME + """</b>
            </p>
        """),
        instructions,
        widgets.HTML("<div style='height: 20px;'></div>"),
        name_input,
        widgets.HTML("<div style='height: 15px;'></div>"),
        download_btn,
        status_output,
        email_section
    ], layout=widgets.Layout(
        padding='30px', 
        border='3px solid #e0e0e0', 
        border_radius='15px',
        background='#fafafa',
        box_shadow='0 4px 16px rgba(0,0,0,0.1)'
    ))
    
    display(main_box)

# =============================================
# EJECUTAR
# =============================================

create_download_and_email_system()

---

## ❓ Preguntas Frecuentes

### ¿Dónde está mi archivo descargado?
Tu archivo se guarda automáticamente en la carpeta **Descargas** de tu computador.

### ¿Necesito crear una cuenta?
**No**, solo necesitas un programa de correo electrónico (que probablemente ya tienes).

### ¿Puedo entregar más de una vez?
Sí, puedes descargar y enviar cuantas veces necesites. Solo asegúrate de indicar en el correo que es una nueva versión.

### ¿Qué pasa si no se abre mi programa de correo?
No hay problema. Envía manualmente un correo a la dirección del profesor con el archivo adjunto.

### ¿Se guarda mi trabajo automáticamente?
Sí, JupyterLite guarda tu trabajo en el navegador automáticamente. Puedes cerrar y volver después.

### ¿Qué formato debe tener mi archivo?
El archivo se descarga automáticamente como `.ipynb`, que es el formato correcto. No lo conviertas a otro formato.

---

## 📧 Contacto

Si tienes problemas técnicos o preguntas sobre la actividad, contacta a:

**Profesor:** [Tu nombre aquí]  
**Email:** profesor@uc.cl  

---

<div style='background: #e8f5e9; padding: 20px; border-radius: 10px; border-left: 5px solid #4caf50; margin-top: 30px;'>
    <h3 style='color: #2e7d32; margin-top: 0;'>✅ ¡Éxito con tu actividad!</h3>
    <p style='margin-bottom: 0; color: #2e7d32;'>
        Recuerda revisar bien tu trabajo antes de entregar. ¡Mucha suerte! 🎓
    </p>
</div>